# !!! DEPRECATED NOTEBOOK !!!

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import seaborn as sns; sns.set()
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

In [2]:
# get the 4th root for embedding: https://developers.googleblog.com/2017/11/introducing-tensorflow-feature-columns.html

## The Preprocessed Data

In [3]:
# the combined forex and economic news data
df = pd.read_csv('fx_with_news.csv', header=[0,1], index_col=0)

In [4]:
df

c_eur                                           \
                         mean       std bar_len bar_spearman bar_log_r   
ctime                                                                    
2016-01-03 22:00:00  1.087467  0.000015    28.0    -0.737459 -0.000037   
2016-01-03 22:05:00  1.087362  0.000049   155.0    -0.181051 -0.000230   
2016-01-03 22:10:00  1.086980  0.000105   138.0    -0.460522 -0.000248   
2016-01-03 22:15:00  1.086938  0.000043   125.0     0.282268 -0.000028   
2016-01-03 22:20:00  1.087057  0.000148   115.0    -0.043127 -0.000120   
...                       ...       ...     ...          ...       ...   
2019-12-31 21:35:00  1.121460  0.000037    62.0    -0.800736 -0.000080   
2019-12-31 21:40:00  1.121460  0.000028    41.0    -0.141550 -0.000009   
2019-12-31 21:45:00  1.121388  0.000046   108.0     0.612689 -0.000018   
2019-12-31 21:50:00  1.121327  0.000046    97.0    -0.912678 -0.000143   
2019-12-31 21:55:00  1.121247  0.000131    91.0     0.075062  0.000294   

                                                                               \
                      first_r     max_r     min_r    last_r bar_quantile_25_r   
ctime                                                                           
2016-01-03 22:00:00  0.000030  0.000030 -0.000016 -0.000007         -0.000009   
2016-01-03 22:05:00  0.000081  0.000081 -0.000149 -0.000149         -0.000029   
2016-01-03 22:10:00  0.000203  0.000203 -0.000147 -0.000046         -0.000064   
2016-01-03 22:15:00 -0.000007  0.000057 -0.000108 -0.000035         -0.000007   
2016-01-03 22:20:00 -0.000144  0.000224 -0.000264 -0.000264         -0.000126   
...                       ...       ...       ...       ...               ...   
2019-12-31 21:35:00  0.000062  0.000062 -0.000036 -0.000018         -0.000027   
2019-12-31 21:40:00 -0.000027  0.000035 -0.000054 -0.000036         -0.000018   
2019-12-31 21:45:00  0.000029  0.000047 -0.000096  0.000011         -0.000025   
2019-12-31 21:50:00  0.000056  0.000092 -0.000122 -0.000086         -0.000015   
2019-12-31 21:55:00 -0.000033  0.000261 -0.000123  0.000261         -0.000078   

                     ...  \
                     ...   
ctime                ...   
2016-01-03 22:00:00  ...   
2016-01-03 22:05:00  ...   
2016-01-03 22:10:00  ...   
2016-01-03 22:15:00  ...   
2016-01-03 22:20:00  ...   
...                  ...   
2019-12-31 21:35:00  ...   
2019-12-31 21:40:00  ...   
2019-12-31 21:45:00  ...   
2019-12-31 21:50:00  ...   
2019-12-31 21:55:00  ...   

                                                             after_counter_ohe  \
                    _united states nondefense capital goods orders ex aircraft   
ctime                                                                            
2016-01-03 22:00:00                                           0.000000           
2016-01-03 22:05:00                                           0.000000           
2016-01-03 22:10:00                                           0.000000           
2016-01-03 22:15:00                                           0.000000           
2016-01-03 22:20:00                                           0.000000           
...                                                                ...           
2019-12-31 21:35:00                                           0.429514           
2019-12-31 21:40:00                                           0.429167           
2019-12-31 21:45:00                                           0.428819           
2019-12-31 21:50:00                                           0.428472           
2019-12-31 21:55:00                                           0.428125           

                                                     \
                    _united states nonfarm payrolls   
ctime                                                 
2016-01-03 22:00:00                             0.0   
2016-01-03 22:05:00                             0.0   
2016-01-03 22:10:00                

In [5]:
# list of currencies
c_list = ['c_eur', 'c_gbp', 'c_jpy']

In [8]:
# make new feature for more stable forecast
for cur in c_list:    
    # logreturn of the means
    df[(cur,'mean_log_r')] = np.log(1.0 + df[(cur,'mean')].pct_change())
    df[(cur,'mean_log_r')] = df[(cur,'mean_log_r')].fillna(0.0)

In [9]:
# column hierarchy
for top in list(df.columns.get_level_values(0).unique()):
    print('----------------------------------')
    print('TOP COLUMN: ', top)
    print('SUBCOLUMNS: ', len(df[top].columns))
    print(list(df[top].columns))


----------------------------------
TOP COLUMN:  c_eur
SUBCOLUMNS:  12
['mean', 'std', 'bar_len', 'bar_spearman', 'bar_log_r', 'first_r', 'max_r', 'min_r', 'last_r', 'bar_quantile_25_r', 'bar_quantile_75_r', 'mean_log_r']
----------------------------------
TOP COLUMN:  c_gbp
SUBCOLUMNS:  12
['mean', 'std', 'bar_len', 'bar_spearman', 'bar_log_r', 'first_r', 'max_r', 'min_r', 'last_r', 'bar_quantile_25_r', 'bar_quantile_75_r', 'mean_log_r']
----------------------------------
TOP COLUMN:  c_jpy
SUBCOLUMNS:  12
['mean', 'std', 'bar_len', 'bar_spearman', 'bar_log_r', 'first_r', 'max_r', 'min_r', 'last_r', 'bar_quantile_25_r', 'bar_quantile_75_r', 'mean_log_r']
----------------------------------
TOP COLUMN:  month
SUBCOLUMNS:  12
['_1', '_2', '_3', '_4', '_5', '_6', '_7', '_8', '_9', '_10', '_11', '_12']
----------------------------------
TOP COLUMN:  dow
SUBCOLUMNS:  6
['_0', '_1', '_2', '_3', '_4', '_6']
----------------------------------
TOP COLUMN:  hour
SUBCOLUMNS:  24
['_0', '_1', '_2',

In [10]:
top_level = list(df.columns.get_level_values(0).unique())
top_level

['c_eur',
 'c_gbp',
 'c_jpy',
 'month',
 'dow',
 'hour',
 'event_cur',
 'event_exist',
 'actual_ohe',
 'surprise_ohe',
 'change_ohe',
 'after_counter_ohe']

## Processing for Model Input

In [11]:
# we will use 2016-2018 as training data, and 2019 as validation and test data
# problem: we have events that occure only in 2019
# we can't normalize/standardize/scale them based on 2016-2018 data
# we could move them to the '_(currency) event' columns, 
# but there is only a few, so here I will delete them

# have we all event type in 2016-2018 yers?
df['event_exist'].loc['2016':'2019'].any().all()

False

In [12]:
# find events occuring only in 2019
event_before_19 = df['event_exist'].loc['2016':'2019'].any()
event_only_19 = list(event_before_19[event_before_19==False].index)
event_only_19

['_european monetary union markit pmi composite',
 '_germany zew survey  economic sentiment',
 '_japan gross domestic product qoq',
 '_united states consumer price index ex food  energy mom',
 '_united states initial jobless claims 4week average',
 '_united states nondefense capital goods orders ex aircraft']

In [13]:
# drop events only occuring in 2019
df.drop(event_only_19, axis=1, level=1, inplace=True)

In [14]:
# have we all event type in 2016-2018 yers?
df['event_exist'].loc['2016':'2019'].any().all()
#ok

True

In [15]:
# we use only train and validation
# Standardizing and normalizing shifts the dataset, where +- sign can be important. Later the model will adjust this by the biases,
# but the information is lost. It could improve the performance if we make a new feature for + and - numbers, but here I don't want to make more features

In [16]:
# spearman doesn't need standardization
c_single_features = ['mean', 'std', 'bar_len']
feature_scalers = dict()

In [17]:
# standardize features based on data up to 2019-01-01 00:00
for cur in c_list:
    for feature in c_single_features:
        column_mean = df[cur, feature].loc['2016':'2019'].mean()
        column_std = df[cur, feature].loc['2016':'2019'].std()
        feature_scalers.update({(cur, feature) : [column_mean, column_std]})
        df.loc[:,(cur, feature)] = (df[cur, feature] - column_mean)/column_std

In [18]:
# !!! later we can use them to rescale if needed
feature_scalers # column_mean, column_std

{('c_eur', 'bar_len'): [412.9294628759021, 381.7685744752972],
 ('c_eur', 'mean'): [1.139248301570798, 0.0499815489615703],
 ('c_eur', 'std'): [0.00010543836669119138, 9.077071115943524e-05],
 ('c_gbp', 'bar_len'): [387.5186401063365, 303.3104183503982],
 ('c_gbp', 'mean'): [1.326336873217585, 0.06648514955410888],
 ('c_gbp', 'std'): [0.000146642848991849, 0.00015890007722327068],
 ('c_jpy', 'bar_len'): [354.61210380295637, 299.81779856625644],
 ('c_jpy', 'mean'): [0.9066954709408044, 0.03345136241766339],
 ('c_jpy', 'std'): [9.800579913539747e-05, 9.589094629696171e-05]}

In [19]:
c_related_features = ['mean_log_r','bar_log_r','first_r','max_r','min_r','last_r','bar_quantile_25_r','bar_quantile_75_r']
# these are related feature, we can think of them as one feature with subfeatures
# they are on similar scale 
# and the pairs on the other side of the mean (like max_r and min_r) are almost symmetric
# to preserve the signs we don't subtract the mean, only divide by the std
# this way we preserve their relative size
# *10000000 is only for easier comparison
print(df['c_eur'][c_related_features].abs().mean()*10000000)
print(df['c_eur'][c_related_features].mean()*10000000)

mean_log_r           1435.641469
bar_log_r            1756.307094
first_r              1079.158746
max_r                1680.077984
min_r                1686.788765
last_r                960.660734
bar_quantile_25_r     647.482120
bar_quantile_75_r     647.172621
dtype: float64
mean_log_r              1.023680
bar_log_r               8.380970
first_r                -0.840415
max_r                1680.077984
min_r               -1686.788765
last_r                  7.540555
bar_quantile_25_r    -647.041955
bar_quantile_75_r     646.659518
dtype: float64


In [20]:
related_std = df[cur, 'bar_log_r'].loc['2016':'2019'].std()

for cur in c_list:
    for feature in c_related_features:
        df.loc[:,(cur, feature)] = (df[cur, feature])/related_std

In [21]:
# !!! needed later to rescale outputs of log_r features
related_std

0.0003561751218763309

In [22]:
# ok, these features remained on similar scale, but the other features as well
print(df['c_eur'][c_related_features].abs().mean())
print(df['c_eur'][c_related_features].mean())

mean_log_r           0.403072
bar_log_r            0.493102
first_r              0.302985
max_r                0.471700
min_r                0.473584
last_r               0.269716
bar_quantile_25_r    0.181788
bar_quantile_75_r    0.181701
dtype: float64
mean_log_r           0.000287
bar_log_r            0.002353
first_r             -0.000236
max_r                0.471700
min_r               -0.473584
last_r               0.002117
bar_quantile_25_r   -0.181664
bar_quantile_75_r    0.181557
dtype: float64


In [23]:
# 'event_cur' is on similar scale, we don't touch (normalization would be better than standardization)
df['event_cur'].max()

_EUR    2.000000
_GBP    2.828427
_JPY    1.732051
_USD    3.000000
dtype: float64

In [26]:
# rescaling output isn't always important, 
# but if the model has different output features it helps to weight their effect on the loss function
# or we can use multiple loss functions and manually set the weight
df_label = df.xs('mean_log_r', axis=1, level=1, drop_level=False)

In [27]:
df_label

,c_eur,c_gbp,c_jpy
,mean_log_r,mean_log_r,mean_log_r
ctime,,,
2016-01-03 22:00:00,0.000000,0.000000,0.000000
2016-01-03 22:05:00,-0.271969,0.959393,-0.038811
2016-01-03 22:10:00,-0.986573,0.692598,-0.022978
2016-01-03 22:15:00,-0.108544,-0.684034,-0.222788
2016-01-03 22:20:00,0.306975,-0.096665,-0.160975
...,...,...,...
2019-12-31 21:35:00,-0.048386,0.197334,-0.022041
2019-12-31 21:40:00,-0.000601,1.112390,0.169539


In [28]:
# should consider every event type alone, but there are 76 event types, so just normalize
df['actual_ohe'] = (df['actual_ohe'] - df['actual_ohe'].loc['2016':'2019'].min())/(df['actual_ohe'].loc['2016':'2019'].max() - df['actual_ohe'].loc['2016':'2019'].min())
# where max-min == 0 we get NaN, fill with 0
df['actual_ohe'] = df['actual_ohe'].fillna(0.0)

In [29]:
# dividing with the max of absolute values keeps the signs
df['surprise_ohe'] = df['surprise_ohe']/df['surprise_ohe'].loc['2016':'2019'].abs().max()
df['surprise_ohe'] = df['surprise_ohe'].fillna(0.0)

df['change_ohe'] = df['change_ohe']/df['change_ohe'].loc['2016':'2019'].abs().max()
df['change_ohe'] = df['change_ohe'].fillna(0.0)

df['after_counter_ohe'] = df['after_counter_ohe']/df['after_counter_ohe'].loc['2016':'2019'].abs().max()
df['after_counter_ohe'] = df['after_counter_ohe'].fillna(0.0)

In [30]:
# df_label = df_label.astype('float16')
# df = df.astype('float16')
df_valid = df.loc['2019':]
df = df.loc[:'2019']
df_label_valid = df_label.loc['2019':]
df_label = df_label.loc[:'2019']

## The Datasets

In [52]:
train_size=288
target_size=10 # this is defined by the network, and not the network is defined by this
shift=1
batch_size=64

In [53]:
# label data begins from the end of the first train data

def ds_input(df, concat_tops, feature_to_last):
    nx = df.to_numpy()
    if (isinstance(df.columns, pd.core.index.MultiIndex) and concat_tops==False):
        top_level_nb = len(df.columns.get_level_values(0).unique())
        sub_level_nb = len(df.columns.get_level_values(1).unique())
        nx = nx.reshape(-1,top_level_nb,sub_level_nb)
        if feature_to_last == True:
            nx = np.moveaxis(nx, [0,1,2], [0,2,1])
        nx = np.squeeze(nx)
    return nx

def make_ds(batch_size, time_steps, shift, skip_steps, df, concat_tops=False, feature_to_last=True):
    nx = ds_input(df, concat_tops, feature_to_last)
    ds = tf.data.Dataset.from_tensor_slices(nx[skip_steps:])
    ds = ds.window(time_steps, shift=shift, drop_remainder=True)
    ds = ds.flat_map(lambda window: window.batch(time_steps))
    return ds.batch(batch_size, drop_remainder=True)

In [54]:
label_dataset_args = {'batch_size':batch_size,
                      'time_steps':target_size,
                      'shift':shift, 
                      'skip_steps':train_size}

ds_label = make_ds(**label_dataset_args, df=df_label)
#ds_label_mr = make_ds(**label_dataset_args, df=df_label, concat_tops=True)
#ds_label_means = make_ds(**label_dataset_args, df=df_label.xs('mean', axis=1, level=1))

ds_label_valid = make_ds(**label_dataset_args, df=df_label_valid)
#ds_label_mr_valid = make_ds(**label_dataset_args, df=df_label_valid, concat_tops=True)
#ds_label_means_valid = make_ds(**label_dataset_args, df=df_label_valid.xs('mean', axis=1, level=1))

In [55]:
input_dataset_args = {'batch_size':batch_size,
                      'time_steps':train_size,
                      'shift':shift, 
                      'skip_steps':0}

ds_bars_feat_end = make_ds(**input_dataset_args, df=df[['c_eur', 'c_gbp', 'c_jpy']])
ds_bars_curr_end = make_ds(**input_dataset_args, df=df[['c_eur', 'c_gbp', 'c_jpy']], concat_tops=False, feature_to_last=False)
ds_bars_simple = make_ds(**input_dataset_args, df=df[['c_eur', 'c_gbp', 'c_jpy']], concat_tops=True)
ds_event_ohe = make_ds(**input_dataset_args, df=df[['actual_ohe','surprise_ohe','change_ohe','after_counter_ohe']], concat_tops=True) # for simplicity
ds_event_exist = make_ds(**input_dataset_args, df=df['event_exist'])
ds_time_cur = make_ds(**input_dataset_args, df=df[['month', 'dow', 'hour', 'event_cur']], concat_tops=True)

ds_bars_feat_end_valid = make_ds(**input_dataset_args, df=df_valid[['c_eur', 'c_gbp', 'c_jpy']])
ds_bars_curr_end_valid = make_ds(**input_dataset_args, df=df_valid[['c_eur', 'c_gbp', 'c_jpy']], concat_tops=False, feature_to_last=False)
ds_bars_simple_valid = make_ds(**input_dataset_args, df=df_valid[['c_eur', 'c_gbp', 'c_jpy']], concat_tops=True)
ds_event_ohe_valid = make_ds(**input_dataset_args, df=df_valid[['actual_ohe','surprise_ohe','change_ohe','after_counter_ohe']], concat_tops=True) # for simplicity
ds_event_exist_valid = make_ds(**input_dataset_args, df=df_valid['event_exist'])
ds_time_cur_valid = make_ds(**input_dataset_args, df=df_valid[['month', 'dow', 'hour', 'event_cur']], concat_tops=True)

In [56]:
future_dataset_args = {'batch_size':batch_size,
                      'time_steps':target_size,
                      'shift':shift, 
                      'skip_steps':train_size}

ds_event_future = make_ds(**future_dataset_args, df=df['event_exist'])
ds_time_curr_future = make_ds(**future_dataset_args, df=df[['month', 'dow', 'hour', 'event_cur']], concat_tops=True)

ds_event_future_valid = make_ds(**future_dataset_args, df=df_valid['event_exist'])
ds_time_curr_future_valid = make_ds(**future_dataset_args, df=df_valid[['month', 'dow', 'hour', 'event_cur']], concat_tops=True)

In [57]:
print('ds_label ',ds_label)

print('ds_bars_feat_end ',ds_bars_feat_end) # not used
print('ds_bars_curr_end ',ds_bars_curr_end) # not used
print('ds_bars_simple ',ds_bars_simple)
print('ds_event_ohe ', ds_event_ohe) 
# more than one ocasion per number => no entity embedding
print('ds_time_cur', ds_time_cur) 
print('ds_event_exist ', ds_event_exist)
# 
print('ds_event_future ', ds_event_future)
print('ds_time_curr_future ', ds_time_curr_future)

ds_label  <BatchDataset shapes: (64, None, 3), types: tf.float64>
ds_bars_feat_end  <BatchDataset shapes: (64, None, 12, 3), types: tf.float64>
ds_bars_curr_end  <BatchDataset shapes: (64, None, 3, 12), types: tf.float64>
ds_bars_simple  <BatchDataset shapes: (64, None, 36), types: tf.float64>
ds_event_ohe  <BatchDataset shapes: (64, None, 280), types: tf.float64>
ds_time_cur <BatchDataset shapes: (64, None, 46), types: tf.float64>
ds_event_exist  <BatchDataset shapes: (64, None, 70), types: tf.float64>
ds_event_future  <BatchDataset shapes: (64, None, 70), types: tf.float64>
ds_time_curr_future  <BatchDataset shapes: (64, None, 46), types: tf.float64>


## Building the Model

In [58]:
class GatedActivationUnit(keras.layers.Layer):
    def __init__(self, activation="tanh", **kwargs):
        super().__init__(**kwargs)
        self.activation = keras.activations.get(activation)
        
    def get_config(self):
        config = super().get_config().copy()
        config.update({
            'activation': self.activation,
        })
        return config 
    
    def call(self, inputs):
        n_filters = inputs.shape[-1] // 2
        linear_output = self.activation(inputs[..., :n_filters])
        gate = keras.activations.sigmoid(inputs[..., n_filters:])
        return self.activation(linear_output) * gate

In [59]:
def wavenet_residual_block(inputs, n_filters, dilation_rate):
    z = keras.layers.Conv1D(2 * n_filters, kernel_size=2, padding="causal",
                            dilation_rate=dilation_rate)(inputs)
    z = GatedActivationUnit()(z)
    z = keras.layers.Conv1D(n_filters, kernel_size=1)(z)
    return keras.layers.Add()([z, inputs]), z

In [60]:
# inputs
input_c1 = keras.layers.Input(shape=[train_size, 12, 3], name = 'input_c1')
input_c2 = keras.layers.Input(shape=[train_size, 3, 12], name = 'input_c2')
input_simple_c = keras.layers.Input(shape=[train_size, 36], name = 'input_simple_c')
input_ohe = keras.layers.Input(shape=[train_size, 280], name = 'input_ohe')
input_time_cur = keras.layers.Input(shape=[train_size, 46], name = 'input_time_cur')
input_event_exist = keras.layers.Input(shape=[train_size, 70], name = 'input_event_exist')

input_f_time_cur = keras.layers.Input(shape=[target_size, 46], name = 'input_f_time_cur')
input_f_event_exist = keras.layers.Input(shape=[target_size, 70], name = 'input_f_event_exist')

In [61]:
def model_event_exist_setup(inputs, embed_dim, name=None):
    out = keras.layers.Conv1D(embed_dim, kernel_size=1, strides=1, padding='valid')(inputs)
    return keras.models.Model(inputs=[inputs], outputs=[out])

model_event_exist = model_event_exist_setup(input_event_exist, 10) #, name='mod_event_exist')
# model_event_exist_f = model_event_exist_setup(input_f_event_exist, 10, name='mod_event_exist_f')
model_event_exist.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_event_exist (InputLaye [(None, 288, 70)]         0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 288, 10)           710       
Total params: 710
Trainable params: 710
Non-trainable params: 0
_________________________________________________________________


In [62]:
def model_time_cur_setup(inputs, embed_dim, name=None):
    out = keras.layers.Conv1D(embed_dim, kernel_size=1, strides=1, padding='valid')(inputs)
    return keras.models.Model(inputs=[inputs], outputs=[out])

model_time_cur = model_time_cur_setup(input_time_cur, 6) #, name='mod_time_cur')
# model_time_cur_f = model_time_cur_setup(input_f_time_cur, 6, name='mod_time_cur_f')
model_time_cur.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_time_cur (InputLayer)  [(None, 288, 46)]         0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 288, 6)            282       
Total params: 282
Trainable params: 282
Non-trainable params: 0
_________________________________________________________________


In [63]:
def model_event_ohe_setup(inputs, embed_dim): 
    out = keras.layers.Conv1D(embed_dim, kernel_size=1, strides=1, padding='valid')(inputs)
    return keras.models.Model(inputs=[inputs], outputs=[out]) #, name = 'mod_event_ohe')

model_event_ohe = model_event_ohe_setup(input_ohe, 16)
model_event_ohe.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_ohe (InputLayer)       [(None, 288, 280)]        0         
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 288, 16)           4496      
Total params: 4,496
Trainable params: 4,496
Non-trainable params: 0
_________________________________________________________________


In [64]:
def wavenet_model_setup(n_layers_per_block, n_blocks, n_filters, n_outputs, feature_dim, name):
    # n_layers_per_block = 10 in the paper
    # n_blocks = 3 in the paper
    # n_filters = 128 in the paper
    # n_outputs = 256 in the paper
    
    inputs = keras.layers.Input(shape=[train_size, feature_dim]) # 164 with cross model
    z = keras.layers.Conv1D(n_filters, kernel_size=2, padding="causal")(inputs)
    skip_to_last = []
    for dilation_rate in [2**i for i in range(n_layers_per_block)] * n_blocks:
        z, skip = wavenet_residual_block(z, n_filters, dilation_rate)
        skip_to_last.append(skip)
    z = keras.activations.relu(keras.layers.Add()(skip_to_last))
    z = keras.layers.Conv1D(n_filters, kernel_size=1, activation="relu")(z)
    Y_proba = keras.layers.Conv1D(n_outputs, kernel_size=1, activation="softmax")(z)
    return keras.models.Model(inputs=[inputs], outputs=[Y_proba], name = name)

model_wn = wavenet_model_setup(n_layers_per_block=3, n_blocks=2, n_filters=64, n_outputs=64, feature_dim=68, name='mod_wavenet_fl32_fd65')
# (fast test) #  model_wn = wavenet_model(n_layers_per_block=2, n_blocks=1, n_filters=1, n_outputs=3)

#keras.utils.plot_model(model, show_shapes=True)
model_wn.summary()

Model: "mod_wavenet_fl32_fd65"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 288, 68)]    0                                            
__________________________________________________________________________________________________
conv1d_5 (Conv1D)               (None, 288, 64)      8768        input_1[0][0]                    
__________________________________________________________________________________________________
conv1d_6 (Conv1D)               (None, 288, 128)     16512       conv1d_5[0][0]                   
__________________________________________________________________________________________________
gated_activation_unit (GatedAct (None, 288, 64)      0           conv1d_6[0][0]                   
______________________________________________________________________________

In [65]:
def time_transformer_model_setup(input_dim, output_dim):
    inputs = keras.layers.Input(shape=[train_size, input_dim])
    z = keras.layers.BatchNormalization()(inputs)
    z = keras.layers.Conv1D(8, kernel_size=5, strides=3, padding='valid', activation='relu')(z)
    z = keras.layers.BatchNormalization()(z)
    z = keras.layers.Conv1D(16, kernel_size=3, strides=3, padding='valid', activation='relu')(z)
    z = keras.layers.BatchNormalization()(z)
    z = keras.layers.Conv1D(output_dim, kernel_size=3, strides=3, padding='valid', activation='relu')(z)
    out = keras.layers.BatchNormalization()(z)
    return keras.models.Model(inputs=[inputs], outputs=[out])

model_time_transformer = time_transformer_model_setup(64, 32)
model_time_transformer.summary()

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 288, 64)]         0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 288, 64)           256       
_________________________________________________________________
conv1d_20 (Conv1D)           (None, 95, 8)             2568      
_________________________________________________________________
batch_normalization_2 (Batch (None, 95, 8)             32        
_________________________________________________________________
conv1d_21 (Conv1D)           (None, 31, 16)            400       
_________________________________________________________________
batch_normalization_3 (Batch (None, 31, 16)            64        
_________________________________________________________________
conv1d_22 (Conv1D)           (None, 10, 32)            1568

## Train Simplest model

In [66]:
def mean_metric(time_start=None, time_end=None, feat_start=None, feat_end=None):
    def mae_segment(y_true, y_pred):
        return tf.abs(y_true[:, time_start:time_end, feat_start:feat_end] - y_pred[:, time_start:time_end, feat_start:feat_end])
    return mae_segment

def mae_mean_full(y_true, y_pred):
    return tf.abs(y_true[:, :, :3] - y_pred[:, :, :3])

def mae_mean_first(y_true, y_pred):
    return tf.abs(y_true[:, 0, :3] - y_pred[:, 0, :3])

def weighted_mae_loss_setup(steps):
    def weighted_mae_loss(y_true, y_pred):
        i_divider = tf.math.sqrt(tf.range(1, steps+1, dtype='float32'))
        loss = tf.abs(y_true - y_pred)
        return tf.math.divide(loss, i_divider[:,None])
    return weighted_mae_loss

In [67]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

In [68]:
ohe = model_event_ohe(input_ohe)
time_cur = model_time_cur(input_time_cur)
event_ex = model_event_exist(input_event_exist)

inputs = keras.layers.Concatenate(axis=-1)([input_simple_c, ohe, time_cur, event_ex])
wave = model_wn(inputs)
out_head = model_time_transformer(wave)

time_cur_f = model_time_cur(input_f_time_cur)
event_ex_f = model_event_exist(input_f_event_exist)

input_f = keras.layers.Concatenate(axis=-1)([out_head, time_cur_f, event_ex_f])
input_f = keras.layers.Conv1D(24, kernel_size=1, strides=1, padding='valid', activation='relu')(input_f)
input_f = keras.layers.BatchNormalization()(input_f)
out_f = keras.layers.Conv1D(3, kernel_size=1, strides=1, padding='valid', activation=None)(input_f)

model_sum = keras.models.Model(inputs=[input_simple_c, input_ohe, input_time_cur, input_event_exist, input_f_time_cur, input_f_event_exist], outputs=[out_f])

model_sum.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ohe (InputLayer)          [(None, 288, 280)]   0                                            
__________________________________________________________________________________________________
input_time_cur (InputLayer)     [(None, 288, 46)]    0                                            
__________________________________________________________________________________________________
input_event_exist (InputLayer)  [(None, 288, 70)]    0                                            
__________________________________________________________________________________________________
input_simple_c (InputLayer)     [(None, 288, 36)]    0                                            
______________________________________________________________________________________________

In [69]:
ds_train = tf.data.Dataset.zip(((ds_bars_simple, ds_event_ohe, ds_time_cur, ds_event_exist, ds_time_curr_future, ds_event_future), 
                                 ds_label_means)).prefetch(tf.data.experimental.AUTOTUNE)
ds_valid = tf.data.Dataset.zip(((ds_bars_simple_valid, ds_event_ohe_valid, ds_time_cur_valid, ds_event_exist_valid, ds_time_curr_future_valid, ds_event_future_valid), 
                                 ds_label_means_valid)).prefetch(tf.data.experimental.AUTOTUNE)

In [70]:
optimizer = keras.optimizers.Adam(learning_rate=0.001)
model_sum.compile(loss=keras.losses.Huber(), optimizer=optimizer, metrics=["mae", mae_mean_first, mae_mean_full])
checkpoint_cb = keras.callbacks.ModelCheckpoint("model_wn_simple.h5", save_best_only=True)

history = model_sum.fit(ds_train, epochs=14,
                       validation_data=ds_valid,
                       callbacks=[checkpoint_cb])

Epoch 1/14
   3498/Unknown - 660s 189ms/step - loss: 0.2229 - mae: 0.4997 - mae_mean_first: 0.5006 - mae_mean_full: 0.4997WARNING:tensorflow:Model was constructed with shape Tensor("input_event_exist:0", shape=(None, 288, 70), dtype=float32) for input (None, 288, 70), but it was re-called on a Tensor with incompatible shape (64, 10, 70).
3498/3498 [==============================] - 869s 248ms/step - loss: 0.2229 - mae: 0.4997 - mae_mean_first: 0.5006 - mae_mean_full: 0.4997 - val_loss: 0.1303 - val_mae: 0.3570 - val_mae_mean_first: 0.3572 - val_mae_mean_full: 0.3570
Epoch 2/14
3498/3498 [==============================] - 871s 249ms/step - loss: 0.2189 - mae: 0.4940 - mae_mean_first: 0.4942 - mae_mean_full: 0.4940 - val_loss: 1.5218 - val_mae: 1.9506 - val_mae_mean_first: 2.0341 - val_mae_mean_full: 1.9506
Epoch 3/14
3498/3498 [==============================] - 891s 255ms/step - loss: 0.2189 - mae: 0.4939 - mae_mean_first: 0.4941 - mae_mean_full: 0.4939 - val_loss: 0.1310 - val_mae: 0.3

KeyboardInterrupt: 

## Train with price and return loss

In [ ]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
inputs = keras.layers.Concatenate(axis=-1)([input_simple_c, 
                                            model_event_ohe(input_ohe), 
                                            model_time_cur(input_time_cur), 
                                            model_event_exist(input_event_exist)])

wave = model_wn(inputs)
out_head = model_time_transformer(wave)

input_f = keras.layers.Concatenate(axis=-1)([out_head, 
                                             model_time_cur_f(input_f_time_cur), 
                                             model_event_exist_f(input_f_event_exist)])

input_f = keras.layers.Conv1D(24, kernel_size=1, strides=1, padding='valid', activation='relu')(input_f)
input_f = keras.layers.BatchNormalization()(input_f)
out_f = keras.layers.Conv1D(6, kernel_size=1, strides=1, padding='valid', activation=None)(input_f)

model_sum = keras.models.Model(inputs=[input_simple_c, input_ohe, input_time_cur, input_event_exist, input_f_time_cur, input_f_event_exist], outputs=[out_f])

model_sum.summary()

In [ ]:
ds_train = tf.data.Dataset.zip(((ds_bars_simple, ds_event_ohe, ds_time_cur, ds_event_exist, ds_time_curr_future, ds_event_future), 
                                 ds_label_mr)).prefetch(tf.data.experimental.AUTOTUNE)
ds_valid = tf.data.Dataset.zip(((ds_bars_simple_valid, ds_event_ohe_valid, ds_time_cur_valid, ds_event_exist_valid, ds_time_curr_future_valid, ds_event_future_valid), 
                                 ds_label_mr_valid)).prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
optimizer = keras.optimizers.Adam(lr=0.001)
model_sum.compile(loss=weighted_mae_loss_setup(target_size), optimizer=optimizer, metrics=["mae", mae_mean_hour, mae_mean_full])
checkpoint_cb = keras.callbacks.ModelCheckpoint("model_wn_out_mr.h5", save_best_only=True)

history = model_sum.fit(ds_train, epochs=14,
                       validation_data=ds_valid,
                       callbacks=[checkpoint_cb])

## Evaluate Model

In [71]:
model_direct = keras.models.load_model("model_wn_simple.h5", custom_objects={'GatedActivationUnit' : GatedActivationUnit, 'mae_mean_full' : mae_mean_full, 'mae_mean_first' : mae_mean_first})

In [ ]:
model_direct.evaluate(ds_valid)

In [76]:
pred = model_sum.predict(ds_valid)

KeyboardInterrupt: 

In [ ]:
pred.shape

## OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO

## Train Simple with decoupled features

In [ ]:
ds_train = tf.data.Dataset.zip(((ds_bars_simple, ds_event_ohe, ds_time_cur, ds_event_exist, ds_time_curr_future, ds_event_future), 
                                 ds_label_means)).prefetch(tf.data.experimental.AUTOTUNE)
ds_valid = tf.data.Dataset.zip(((ds_bars_simple_valid, ds_event_ohe_valid, ds_time_cur_valid, ds_event_exist_valid, ds_time_curr_future_valid, ds_event_future_valid), 
                                 ds_label_means_valid)).prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
# future and past embeds are different
model_time_cur_f = model_time_cur_setup(input_f_time_cur, 6, name='model_time_cur_f')
model_event_exist_f = model_event_exist_setup(input_f_event_exist, 10, name='model_event_exist_f')

In [ ]:
ohe = model_event_ohe(input_ohe)
time_cur = model_time_cur(input_time_cur)
event_ex = model_event_exist(input_event_exist)
inputs = keras.layers.Concatenate(axis=-1)([input_simple_c, ohe, time_cur, event_ex])
wave = model_wn(inputs)
out_head = model_time_transform(wave)

time_cur_f = model_time_cur_f(input_f_time_cur)
event_ex_f = model_event_exist_f(input_f_event_exist)

input_f = keras.layers.Concatenate(axis=-1)([out_head, time_cur_f, event_ex_f])
out_f = keras.layers.Conv1D(3, kernel_size=1, strides=1, padding='valid')(input_f)

model_sum = keras.models.Model(inputs=[input_simple_c, input_ohe, input_time_cur, input_event_exist, input_f_time_cur, input_f_event_exist], outputs=[out_f])

model_sum.summary()

In [ ]:
optimizer = keras.optimizers.Adam(lr=0.001)
model_sum.compile(loss=keras.losses.Huber(), optimizer=optimizer, metrics=["mae", mean_metric(last_index=None)])
checkpoint_cb = keras.callbacks.ModelCheckpoint("model_wn_decoup.h5", save_best_only=True)

history = model_sum.fit(ds_train, epochs=14,
                       validation_data=ds_valid,
                       callbacks=[checkpoint_cb])

## Redefine Head Model

In [ ]:
def head_model2_setup():
    inputs = keras.layers.Input(shape=[train_size, 3])
    z = keras.layers.BatchNormalization()(inputs)
    z = keras.layers.Conv1D(8, kernel_size=5, strides=3, padding='valid', activation='relu')(z)
    z = keras.layers.BatchNormalization()(z)
    z = keras.layers.Conv1D(16, kernel_size=3, strides=2, padding='valid', activation='relu')(z)
    z = keras.layers.BatchNormalization()(z)
    z = keras.layers.Conv1D(32, kernel_size=3, strides=2, padding='valid', activation='relu')(z)
    out = keras.layers.BatchNormalization()(z)
    return keras.models.Model(inputs=[inputs], outputs=[out], name = 'model_head2')

model_head2 = head_model2_setup()
model_head2.summary()

In [ ]:
# bottleneck?
model_event_ohe = model_event_ohe_setup(input_ohe, 64)
model_event_ohe.summary()

In [ ]:
# +48 features
model_wn = wavenet_model_setup(n_layers_per_block=3, n_blocks=2, n_filters=64, n_outputs=3, feature_dim=113, name='model_wn_fl64_fd81')
# model_wn_out = wavenet_model_setup(n_layers_per_block=3, n_blocks=1, n_filters=64, n_outputs=3, feature_dim=48, name='model_wn_out_fl64_fd_48')

In [ ]:
target_size=71

In [ ]:
ds_label = make_ds(batch_size=batch_size,
                   time_steps=target_size, 
                   shift=shift, 
                   skip_steps=train_size, 
                   df=df_label)

ds_label_valid = make_ds(batch_size=batch_size,
                   time_steps=target_size, 
                   shift=shift, 
                   skip_steps=train_size, 
                   df=df_label_valid)
ds_event_future = make_ds(  batch_size=batch_size,
                            time_steps=target_size, 
                            shift=shift, 
                            skip_steps=train_size, 
                            df=df['event_exist'])
ds_time_curr_future = make_ds(  batch_size=batch_size,
                            time_steps=target_size, 
                            shift=shift, 
                            skip_steps=train_size, 
                            df=df[['month', 'dow', 'hour', 'event_cur']], concat_tops=True)

ds_event_future_valid = make_ds(  batch_size=batch_size,
                            time_steps=target_size, 
                            shift=shift, 
                            skip_steps=train_size, 
                            df=df_valid['event_exist'])
ds_time_curr_future_valid = make_ds(  batch_size=batch_size,
                            time_steps=target_size, 
                            shift=shift, 
                            skip_steps=train_size, 
                            df=df_valid[['month', 'dow', 'hour', 'event_cur']], concat_tops=True)

input_f_time_cur = keras.layers.Input(shape=[target_size, 46], name = 'input_f_time_cur')
input_f_event_exist = keras.layers.Input(shape=[target_size, 70], name = 'input_f_event_exist')

In [ ]:
ohe = model_event_ohe(input_ohe)
time_cur = model_time_cur(input_time_cur)
event_ex = model_event_exist(input_event_exist)
inputs = keras.layers.Concatenate(axis=-1)([input_simple_c, ohe, time_cur, event_ex])
wave = model_wn(inputs)
out_head = model_head2(wave)

time_cur_f = model_time_cur(input_f_time_cur)
event_ex_f = model_event_exist(input_f_event_exist)

input_f = keras.layers.Concatenate(axis=-1)([out_head, time_cur_f, event_ex_f])
input_f = keras.layers.Conv1D(48, kernel_size=3, strides=1, padding='causal', activation='relu')(input_f)
out_f = keras.layers.Conv1D(3, kernel_size=1, strides=1, padding='valid')(input_f)

model_sum = keras.models.Model(inputs=[input_simple_c, input_ohe, input_time_cur, input_event_exist, input_f_time_cur, input_f_event_exist], outputs=[out_f])

model_sum.summary()

In [ ]:
# test if model converge
'''
ds_worker_test = tf.data.Dataset.zip(((ds_bars_simple, ds_event_ohe, ds_time_cur, ds_event_exist, ds_time_curr_future, ds_event_future), 
                                 ds_label)).shuffle(64).prefetch(tf.data.experimental.AUTOTUNE).take(64)
optimizer = keras.optimizers.Adam(lr=0.001)
model_sum.compile(loss=keras.losses.Huber(), optimizer=optimizer, metrics=["mae"])
history = model_sum.fit(ds_worker_test, epochs=3)
'''

In [ ]:
ds_train = tf.data.Dataset.zip(((ds_bars_simple, ds_event_ohe, ds_time_cur, ds_event_exist, ds_time_curr_future, ds_event_future), 
                                 ds_label)).prefetch(tf.data.experimental.AUTOTUNE)
ds_valid = tf.data.Dataset.zip(((ds_bars_simple_valid, ds_event_ohe_valid, ds_time_cur_valid, ds_event_exist_valid, ds_time_curr_future_valid, ds_event_future_valid), 
                                 ds_label_valid)).prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
optimizer = keras.optimizers.Adam(lr=0.001)
model_sum.compile(loss=keras.losses.Huber(), optimizer=optimizer, metrics=["mae"])
checkpoint_cb = keras.callbacks.ModelCheckpoint("model_wn_64_head_2.h5", save_best_only=True)

history = model_sum.fit(ds_train, epochs=20,
                       validation_data=ds_valid,
                       callbacks=[checkpoint_cb])

# Train Cross Feature model

In [ ]:
# 
z1 = keras.layers.Conv2D(2*3, kernel_size=1, padding="valid")(input_c1)
z1 = keras.layers.Reshape([train_size,66])(z1)

z2 = keras.layers.Conv2D(2*11, kernel_size=1, padding="valid")(input_c2)
z2 = keras.layers.Reshape([train_size,66])(z2)

out = keras.layers.Concatenate(axis=-1)([z1, z2])
model_cur_cross_in = keras.models.Model(inputs=[input_c1, input_c2], outputs=[out])
model_cur_cross_in._name = 'model_cur_cross_in'

model_cur_cross_in.summary()

In [ ]:
model_wn = wavenet_model(n_layers_per_block=3, n_blocks=2, n_filters=256, n_outputs=3, feature_dim=164)
# (fast test) #  model_wn = wavenet_model(n_layers_per_block=2, n_blocks=1, n_filters=1, n_outputs=3)
model_wn._name = 'model_wavenet'

#keras.utils.plot_model(model, show_shapes=True)
model_wn.summary()

In [ ]:
ohe = model_event_ohe(input_ohe)
time_cur = model_time_cur(input_time_cur)
event_ex = model_event_exist(input_event_exist)
cur_cross = model_cur_cross_in([input_c1, input_c2])
inputs = keras.layers.Concatenate(axis=-1)([cur_cross, ohe, time_cur, event_ex])
wave = model_wn(inputs)
out_head = model_head(wave)

time_cur_f = model_time_cur(input_f_time_cur)
event_ex_f = model_event_exist(input_f_event_exist)

input_f = keras.layers.Concatenate(axis=-1)([out_head, time_cur_f, event_ex_f])
out_f = keras.layers.Conv1D(3, kernel_size=1, strides=1, padding='valid')(input_f)

model_sum = keras.models.Model(inputs=[input_c1, input_c2, input_ohe, input_time_cur, input_event_exist, input_f_time_cur, input_f_event_exist], outputs=[out_f])

model_sum.summary()

In [ ]:
out = keras.layers.Conv1D(64, kernel_size=1, strides=1, padding='valid')(input_ohe)
model_event_ohe = keras.models.Model(inputs=[input_ohe], outputs=[out])
model_event_ohe._name = 'model_event_ohe'

model_event_ohe.summary()

In [ ]:
label_transform = dict()
# standardize features based on data up to 2019-01-01 00:00
for cur in c_list:
    mean_to_19 = df_label[cur, 'mean'].loc['2016':'2019'].mean()
    std_to_19 = df_label[cur, 'mean'].loc['2016':'2019'].std()
    label_transform.update({(cur, 'mean') : {'column_mean' : mean_to_19, 'column_std' : std_to_19}})
    #df_label.loc[:,(cur, 'mean')] = (df_label.loc[:,(cur, 'mean')] - mean_to_19)/std_to_19